<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/BX-gpt-app/gemma_ko_2b_rag_mongodb_embedding_CBP_certi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 설치

In [1]:
!pip install -q datasets pandas pymongo sentence_transformers
!pip install -q -U transformers
!pip install -q accelerate
!pip install -q "pymongo[srv]"
!pip install -q -U bitsandbytes
# !pip install -q PyPDF2 openparse
!pip install -q langchain langchain_experimental langchain_openai
# !pip install -q unstructured > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu

# 2. Dataset

## 마크다운 문서를 binary 모드로 읽어 디코딩

In [2]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

# 파일을 바이너리 모드로 읽고 디코딩
def read_file_with_encoding(file_path, encodings=['utf-8', 'latin-1', 'cp1252']):
    for encoding in encodings:
        try:
            with open(file_path, 'rb') as file:
                binary_data = file.read()
                return binary_data.decode(encoding)
        except UnicodeDecodeError as e:
            print(f"Failed to decode with encoding {encoding}: {e}")
    raise ValueError("Unable to decode the file with provided encodings.")

# 파일 읽기
# markdown_text = read_file_with_encoding(test_doc_md)

# data = [Document(page_content=markdown_text)]
# print(data[0])
# readme_content = data[0].page_content
# print(readme_content[:10000])

## 개행/svg 파일 제거

In [3]:
import re

def remove_empty_lines(text):
    # 여러 개의 개행 문자를 하나의 개행 문자로 대체
    text = re.sub(r'\n\s*\n', '\n', text)
    return text

def remove_image_lines(text):
    # ![...](...svg) 또는 ![...](...png)로 끝나는 라인
    pattern_svg_jpg = r'^.*!\[.*?\]\(.*?\.(svg|png)\).*$'
    # single hash
    pattern_single_hash = r'^# .*$'
    cleaned_text = re.sub(pattern_svg_jpg, '', text, flags=re.MULTILINE)

    lines = cleaned_text.split('\n')
    cleaned_lines = [line for line in lines if not re.match(pattern_single_hash, line)]
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

# cleaned_text = remove_empty_lines(data[0].page_content)
# cleaned_text = remove_image_lines(cleaned_text)
# cleaned_text = remove_empty_lines(cleaned_text)
# print(cleaned_text)

## multi hash/stars 기준 chunking/반복된 hash 제거

In [4]:
def chunk_by_patterns(text):
    # ##, ###, ####, 그리고 **로 시작하고 끝나는 패턴을 기준으로 문서를 청킹
    pattern = r'(\n#{2,4} .*|\n\*\* .* \*\*)'
    # 패턴을 기준으로 분할, 패턴도 포함되도록 분할
    chunks = re.split(pattern, text)
    # 결과 청크를 병합하여 각 청크가 패턴과 내용을 모두 포함하도록 함
    merged_chunks = []
    for i in range(1, len(chunks), 2):
        merged_chunks.append(chunks[i] + chunks[i + 1])
    return merged_chunks

def remove_repeated_triple_hash(chunks):
    cleaned_chunks = []
    for chunk in chunks:
        lines = chunk.split('\n')
        cleaned_lines = []
        triple_hash_count = 0
        for line in lines:
            if line.startswith('###'):
                triple_hash_count += 1
                if triple_hash_count > 1:
                    continue  # 연속된 ### 라인이 5줄 이하인 경우 제거
            else:
                triple_hash_count = 0
            cleaned_lines.append(line)

        cleaned_chunks.append('\n'.join(cleaned_lines))
    return cleaned_chunks

def remove_hashes_and_number_patterns(text):
    # 모든 # 제거 및 숫자.숫자.숫자 형태 제거
    text = text.strip()
    text = re.sub(r'#', '', text)
    text = re.sub(r'\b\d+(\.\d+)+\b', '', text)
    return text

In [5]:
import pandas as pd

df = pd.DataFrame(columns=['text'])

pdf_files = [
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/01-BX-CBP-제품소개.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/12-BX-CBP-공통.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/21-BX-CBP-베이스컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/22-BX-CBP-상품컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/23-BX-CBP-액터컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/24-BX-CBP-계약컴포넌트.md',
    '/content/drive/MyDrive/AI-modeling/BX-gpt-app/25-BX-CBP-정산컴포넌트.md',
]

for pdf_file in pdf_files:
    print(f"Start Processing {pdf_file}...")

    curr_markdown_text = read_file_with_encoding(pdf_file)

    curr_data = [Document(page_content=curr_markdown_text)]
    curr_cleaned_text = remove_empty_lines(curr_data[0].page_content)
    curr_cleaned_text = remove_image_lines(curr_cleaned_text)
    curr_cleaned_text = remove_empty_lines(curr_cleaned_text)
    curr_chunks = chunk_by_patterns(curr_cleaned_text)

    final_chunks = remove_repeated_triple_hash(curr_chunks)

    for i, chunk in enumerate(final_chunks):
      cleaned_chunk = remove_hashes_and_number_patterns(chunk)

      if len(cleaned_chunk.strip().split('\n')) == 1:
        continue

      df = pd.concat([df, pd.DataFrame({'text': [cleaned_chunk]})], ignore_index=True)
      # print(cleaned_chunk)
      # print('@@@@@'*10)

print(df.shape)
print(df.head(5))

Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/01-BX-CBP-제품소개.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/02-BX-PF-제품소개.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/12-BX-CBP-공통.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/21-BX-CBP-베이스컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/22-BX-CBP-상품컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/23-BX-CBP-액터컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/24-BX-CBP-계약컴포넌트.md...
Start Processing /content/drive/MyDrive/AI-modeling/BX-gpt-app/25-BX-CBP-정산컴포넌트.md...
(434, 1)
                                                text
0   . 금융기관 차세대시스템 구축 연혁\n우리나라 은행 IT 시스템은 1970년대 과...
1   . 차세대시스템 구축 방향성\n차세대시스템 구축을 위해 Framework, Pro...
2   . BX CBP History\n이런 Needs에 맞게 뱅크워어글로벌(BwG)은 ...
3   . BwG 코어뱅킹 아키텍처 Overview\nBwG 코어뱅킹

# 3. Embedding

In [11]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")

def get_embedding(text: str) -> list[float]:
    if not text.strip():
      print("empty")
      return []
    return embedding_model.encode(text).tolist()

df["embedding"] = df["text"].apply(get_embedding)
df.head(5)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

# 4. db connection

In [6]:
!curl ipecho.net/plain

34.16.141.17

In [7]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["spider504"]
collection = db["240703_movie_collection"]

Connection to MongoDB successful


In [65]:
collection.delete_many({})

documents = df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


# 5. Vector Search

In [8]:
def vector_search(user_query, collection):
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "text": 1,
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    results = collection.aggregate(pipeline)

    return list(results)

In [13]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"text: {result.get('text', 'N/A')}\n"
        # search_result += f"score: {result.get('score', 'N/A')}\n"
    return search_result

# 6. Prompt

In [18]:
# Conduct query with retrival of sources
query = "BX CBP에서는 신규로 업무서비스를 개발할 때 어떤 개발 방식을 적용하나요?"
source_information = get_search_result(query, collection)

combined_information = (
f"""
Query: {query}
Continue to answer the query by using the Search Results.
Please refer to the following information in order of priority. The information at the top is more important.
You should answer with Korean.

{source_information}
"""
)

print(combined_information)


Query: BX CBP에서는 신규로 업무서비스를 개발할 때 어떤 개발 방식을 적용하나요?
Continue to answer the query by using the Search Results.
Please refer to the following information in order of priority. The information at the top is more important.
You should answer with Korean.

text:  . 메뉴 처리과정
BX CBP에서 사용하는 화면들의 정보를 관리하고 사용자그룹/채널별로 메뉴를 구성한다. 이렇게 구성된 메뉴는 역할 화면 권한을 적용하며 로그인 후 시스템 메뉴로 표현된다
text:  . 기능 커버리지
BX CBP는 고객, 수신, 여신, 외환과 같은 코어뱅킹 업무처리를 위한 대부분의 주요 기능을 제공한다.
text:  . 사전의 정의 및 관리 항목
BX CBP에서 사전이란?
* 메타의 기능을 활용하여 관리하는 BX CBP의 리소스 명명 표준
사전으로 관리하는 항목은?
- 단어, 복합단어, 도메인 등의 메타정보를 관리함
- 각 유형별 다국어 정보도 사전의 항목으로 관리함
사전으로 관리하는 항목들의 사용처는 어떤 것들이 있는가?
- 단어(복합단어)들과 도메인을 조합하여 표준속성명을 생성함
  - 표준을 적용한 표준속성명이 생성됨 (속성의 의미와 Data 유형)
  - 표준속성명을 기반으로 생성한 리소스명들에게도 해당 표준이 유지됨
text:  . 기관의 구조
BX CBP는 기관의 본래 의미를 따르면서 모든 Data의 기준이 되는 최상위 개념으로서 표준기관을 관리하고 있다. 또한 표준기관과 설정기관을 통해 멀티기관을 지원한다.




# 7. loading LLM

In [15]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [19]:
raw_input_ids = tokenizer(query, return_tensors="pt").to("cuda")
raw_response = model.generate(**raw_input_ids, max_new_tokens=500)
print('### only Gemma', '\n')
print(tokenizer.decode(raw_response[0]))

input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print('\n', '### Gemma on RAG', '\n')
print(tokenizer.decode(response[0]))

### only Gemma 

<bos>BX CBP에서는 신규로 업무서비스를 개발할 때 어떤 개발 방식을 적용하나요?

**신규 업무서비스 개발 방식의 적용 방식**

**1. 개인 개발 방식**

* 개발자는 개인적으로 업무서비스를 개발하고 테스트합니다.
* 개발 과정에서 다른 개발자와 교환하고 피드백을 주고받습니다.
* 업무서비스 개발 완료 후, 개발자는 업무서비스를 서비스 시작합니다.

**2. 모바일 개발 방식**

* 개발자는 모바일 개발 환경을 사용하여 업무서비스를 개발합니다.
* 모바일 개발 환경은 다양한 모바일 기기에서 동일한 환경을 제공할 수 있도록 설계됩니다.
* 업무서비스 개발 완료 후, 모바일 개발자는 업무서비스를 서비스 시작합니다.

**3. 지속 가능한 개발 방식**

* 개발자는 환경적 측면을 고려하여 업무서비스를 개발합니다.
* 개발 과정에서 에너지 효과적인 기술과 프레임워크를 사용합니다.
* 업무서비스 개발 완료 후, 에너지 효과적인 업무서비스를 서비스 시작합니다.

**4. 혼합 개발 방식**

* 개발자는 개인 개발 방식과 모바일 개발 방식을 결합하여 업무서비스를 개발합니다.
* 개인 개발과 모바일 개발 과정을 병행하여 업무서비스를 빠르게 개발할 수 있습니다.
* 업무서비스 개발 완료 후, 개발자는 업무서비스를 서비스 시작합니다.<eos>

 ### Gemma on RAG 

<bos>
Query: BX CBP에서는 신규로 업무서비스를 개발할 때 어떤 개발 방식을 적용하나요?
Continue to answer the query by using the Search Results.
Please refer to the following information in order of priority. The information at the top is more important.
You should answer with Korean.

text:  . 메뉴 처리과정
BX CBP에서 사용하는 화면들의 정보를 관리하고 사용자그룹/채널별로 